In [16]:
from minio import Minio
from dotenv import load_dotenv
import os
import json

In [17]:
load_dotenv()

minio_client = Minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"), 
    secret_key=os.getenv("MINIO_SECRET_KEY"), 
    secure=False 
)
bucket_name = os.getenv("MINIO_BUCKET_NAME")

In [18]:
import re

# Lista de bairros de Fortaleza/CE
bairros_fortaleza = [
    "Aeroporto", "Aerolândia", "Alagadiço Novo", "Aldeota", "Alto da Balança",
    "Álvaro Weyne", "Amadeu Furtado", "Ancuri", "Antônio Bezerra", "Autran Nunes",
    "Barra do Ceará", "Barroso", "Bela Vista", "Benfica", "Bom Futuro",
    "Bom Jardim", "Bonsucesso", "Cais do Porto", "Cajazeiras", "Cambeba",
    "Canindezinho", "Carlito Pamplona", "Cidade 2000", "Cidade dos Funcionários", "Coaçu",
    "Cocó", "Conjunto Ceará I", "Conjunto Ceará II", "Conjunto Esperança", "Conjunto Palmeiras",
    "Couto Fernandes", "Cristo Redentor", "Curió", "Damas", "Demócrito Rocha",
    "Dendê", "Dias Macedo", "Dionísio Torres", "Dom Lustosa", "Edson Queiroz",
    "Ellery", "Engenheiro Luciano Cavalcante", "Farias Brito", "Fátima", "Floresta",
    "Genibaú", "Granja Lisboa", "Granja Portugal", "Guajeru", "Guararapes",
    "Henrique Jorge", "Itaoca", "Itaperi", "Jacarecanga", "Jangurussu",
    "Jardim América", "Jardim Cearense", "Jardim das Oliveiras", "Jardim Guanabara", "Jardim Iracema",
    "João XXIII", "Joaquim Távora", "Jóquei Clube", "José Bonifácio", "José de Alencar",
    "Lagoa Redonda", "Manuel Dias Branco", "Manoel Sátiro", "Maraponga", "Messejana",
    "Meireles", "Mondubim", "Monte Castelo", "Montese", "Moura Brasil",
    "Mucuripe", "Panamericano", "Papicu", "Parangaba", "Parque Araxá",
    "Parque Dois Irmãos", "Parque Iracema", "Parque Manibura", "Parquelândia", "Parreão",
    "Passaré", "Paupina", "Pedras", "Pici", "Pirambu",
    "Planalto Ayrton Senna", "Praia de Iracema", "Praia do Futuro I", "Praia do Futuro II", "Prefeito José Walter",
    "Presidente Kennedy", "Presidente Vargas", "Quintino Cunha", "Rodolfo Teófilo", "Sabiaguaba",
    "Salinas", "Santa Rosa", "São Bento", "São Gerardo", "São João do Tauape",
    "São José", "Sapiranga", "Serrinha", "Siqueira", "Varjota",
    "Vicente Pinzon", "Vila Ellery", "Vila Peri", "Vila União", "Vila Velha"
]


mensagens_com_bairros = []

In [19]:
import unicodedata
def normalizar_texto(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [20]:
bairros_normalizados = [normalizar_texto(bairro) for bairro in bairros_fortaleza]

In [21]:
bairros_normalizados

['aeroporto',
 'aerolandia',
 'alagadico novo',
 'aldeota',
 'alto da balanca',
 'alvaro weyne',
 'amadeu furtado',
 'ancuri',
 'antonio bezerra',
 'autran nunes',
 'barra do ceara',
 'barroso',
 'bela vista',
 'benfica',
 'bom futuro',
 'bom jardim',
 'bonsucesso',
 'cais do porto',
 'cajazeiras',
 'cambeba',
 'canindezinho',
 'carlito pamplona',
 'cidade 2000',
 'cidade dos funcionarios',
 'coacu',
 'coco',
 'conjunto ceara i',
 'conjunto ceara ii',
 'conjunto esperanca',
 'conjunto palmeiras',
 'couto fernandes',
 'cristo redentor',
 'curio',
 'damas',
 'democrito rocha',
 'dende',
 'dias macedo',
 'dionisio torres',
 'dom lustosa',
 'edson queiroz',
 'ellery',
 'engenheiro luciano cavalcante',
 'farias brito',
 'fatima',
 'floresta',
 'genibau',
 'granja lisboa',
 'granja portugal',
 'guajeru',
 'guararapes',
 'henrique jorge',
 'itaoca',
 'itaperi',
 'jacarecanga',
 'jangurussu',
 'jardim america',
 'jardim cearense',
 'jardim das oliveiras',
 'jardim guanabara',
 'jardim iracema'

In [22]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [23]:
dados = []
list_objects = minio_client.list_objects(bucket_name, prefix='Portalnoticiasceara', recursive=True)
# Processar cada objeto encontrado
for obj in list_objects:
    if obj.object_name.endswith('.json'):  # Verificar se é um arquivo JSON
        print(f"Lendo o arquivo: {obj.object_name}")
        try:
            # Baixar o objeto
            response = minio_client.get_object(bucket_name=bucket_name, object_name=obj.object_name)
            
            # Ler o conteúdo
            data = response.read().decode('utf-8')
            
            # Parsear JSON
            json_data = json.loads(data)
            
            # Verificar se é uma lista
            if isinstance(json_data, list):
                for item in json_data:
                    # Extrair informações relevantes
                    message = item.get('message', '')
                    message_id = item.get('id', None)
                    date = item.get('date', None)
                    media_path = item.get('media_path', None)
                    
                    if message:
                        # Normalizar a mensagem
                        message_normalizada = normalizar_texto(message)
                        
                        # Identificar bairros mencionados
                        bairros_mencionados = [
                            bairro_original for bairro_original, bairro_normalizado in zip(bairros_fortaleza, bairros_normalizados)
                            if bairro_normalizado in message_normalizada
                        ]
                        
                        # Se algum bairro foi identificado, salvar os dados
                        if bairros_mencionados:
                            for bairro in bairros_mencionados:
                                dados.append({
                                    "id": message_id,
                                    "mensagem": message,
                                    "bairro": bairro,
                                    "data": date,
                                    "media_path": media_path
                                })
            
            # Fechar o response
            response.close()
            response.release_conn()
            
            with open('./silver_metadata.json', 'w', encoding='utf-8') as json_file:
                json.dump(dados, json_file, ensure_ascii=False, indent=4)

        except Exception as e:
            print(f"Erro ao processar {obj.object_name}: {e}")

Lendo o arquivo: Portalnoticiasceara/2024-12-14/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-15/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-16/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-17/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-25/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-26/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-27/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-28/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-29/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-30/metadata.json
Lendo o arquivo: Portalnoticiasceara/2024-12-31/metadata.json
Lendo o arquivo: Portalnoticiasceara/2025-01-01/metadata.json


In [24]:
df = pd.DataFrame(dados)

In [25]:
df.tail()

,id,mensagem,bairro,data,media_path
226,6853,"**MOTIVO DOS FOGOS NA COMUNIDADE DA COLÔNIA (GDE) BARRA DO CEARÁ EM FORTALEZA É POR CONTA DO SÉTIMO DIA DE UM TAL DE ""KAKÁ""**",Barra do Ceará,2024-12-31T17:13:57+00:00,None
227,6857,"**Idoso é executado a tiros no bairro Cristo Redentor, em Fortaleza**\n\nNa tarde desta terça-feira, 31, um idoso, ainda sem identificação, foi executado com disparos de arma de fogo por trás da UPA da Leste Oeste, no conjunto Alves de Lima (CV), bairro Cristo Redentor, em Fortaleza.\n\nAté o momento, a motivação do crime é desconhecida. A Polícia Militar esteve no local para isolar a área e iniciar as investigações. O caso será apurado pela Polícia Civil.",Cristo Redentor,2024-12-31T20:31:09+00:00,Portalnoticiasceara/2024-12-31/media/photo_6857.jpg.zstd
228,6870,Planalto Ayrton Senna homicídio nas primeiras horas do primeiro de janeiro de 2025.,Planalto Ayrton Senna,2025-01-01T16:22:21+00:00,Portalnoticiasceara/2025-01-01/media/photo_6870.jpg.zstd
229,6882,"PM VITIMA(3°SGT PM SOUSA/LOTADO NO RAIO BATURITÉ, SUICIDIO POR ARMA DE FOGO NA CASA DOS PAIS- RUA LINEU MACHADO 1292 JOÃO XXIII) FORTALEZA-AIS06 13H15 RAIO04",João XXIII,2025-01-01T17:26:57+00:00,Portalnoticiasceara/2025-01-01/media/photo_6882.jpg.zstd
230,6885,Rua Cerejeiras no Parque Santa Rosa (GDE) em fortaleza,Santa Rosa,2025-01-01T17:51:41+00:00,Portalnoticiasceara/2025-01-01/media/photo_6885.jpg.zstd


In [26]:
df[df['bairro'] == 'Barra do Ceará'].sort_values(by='data', ascending=False)

,id,mensagem,bairro,data,media_path
226,6853,"**MOTIVO DOS FOGOS NA COMUNIDADE DA COLÔNIA (GDE) BARRA DO CEARÁ EM FORTALEZA É POR CONTA DO SÉTIMO DIA DE UM TAL DE ""KAKÁ""**",Barra do Ceará,2024-12-31T17:13:57+00:00,None
214,6731,"**Suspeito de homicídio é encontrado morto na Barra do Ceará, Fortaleza**\n\nNa comunidade do Gueto (CV), no bairro Barra do Ceará, em Fortaleza, um homem, apontado como responsável pela morte de Erick na Rua 6 Companheiros, foi encontrado morto dentro de uma residência.\n\nDe acordo com informações, o suspeito estava comemorando o assassinato de Erick quando foi sequestrado por indivíduos ligados à vítima. Ele foi levado à comunidade, onde foi executado.\n\nA polícia foi acionada e encontrou o corpo do homem, ainda sem identificação, durante uma operação na área.",Barra do Ceará,2024-12-29T22:37:29+00:00,Portalnoticiasceara/2024-12-29/media/video_6731.mp4.zstd
204,6607,**ATENÇÃO 🚨** DISPAROS DE ARMA DE FOGO NESTE EXATO MOMENTO NA COMUNIDADE DA COLÔNIA (GDE) BAIRRO BARRA DO CEARÁ EM FORTALEZA.\n\nSMD\nAMD,Barra do Ceará,2024-12-28T01:02:57+00:00,None
189,6493,"**Homicídio registrado há poucos minutos na Barra do Ceará, em Fortaleza**\n\nUm homicídio foi registrado há poucos minutos na rua peri próximo à Escola Agostinho Moreira,bairro Barra do Ceará, em Fortaleza. A vítima, um homem ainda sem identificação, foi atingida por disparos de arma de fogo, vindo a óbito no local.\n\nEquipes da Polícia Militar já estão no local para realizar os primeiros levantamentos. A motivação e a autoria do crime ainda são desconhecidas e serão investigadas pela Delegacia de Homicídios e Proteção à Pessoa (DHPP).",Barra do Ceará,2024-12-26T20:26:59+00:00,Portalnoticiasceara/2024-12-26/media/photo_6493.jpg.zstd
188,6490,"**ATENÇÃO 🚨** HOMICÍDIO REGISTRADO A POUCOS MINUTOS PRÓXIMO A ESCOLA AGOSTINHO MOREIRA,BAIRRO BARRA DO CEARÁ EM FORTALEZA.\n\nNOSSA EQUIPE ESTÁ COLHENDO MAIS DETALHES. 🗞",Barra do Ceará,2024-12-26T20:06:25+00:00,None
186,6487,**ATENÇÃO 🚨** DISPAROS DE ARMA DE FOGO FORAM REGISTRADOS A POUCOS MINUTOS NA COMUNIDADE DA COLÔNIA (GDE) BARRA DO CEARÁ EM FORTALEZA.\n\nSMD,Barra do Ceará,2024-12-26T19:47:08+00:00,None
185,6479,"**Suspeito tenta fugir da equipe raio pelo mar, mas acaba sendo preso\n**\nNa manhã desta quinta-feira (26) um suspeito tentou fugir da equipe raio próximo ao vila do mar, na barra do Ceará. O mesmo rebolou uma pistola calibre .380 no mar, Logo após a equipe raio acionou outras equipes e obtendo êxito e achando a pistola na água.",Barra do Ceará,2024-12-26T13:31:50+00:00,Portalnoticiasceara/2024-12-26/media/photo_6479.jpg.zstd
105,5852,"Barra do Ceará, fortaleza.\n🚨🚨",Barra do Ceará,2024-12-16T10:35:09+00:00,None
166,5852,"Barra do Ceará, fortaleza.\n🚨🚨",Barra do Ceará,2024-12-16T10:35:09+00:00,None
30,5724,Atualização 🚨\n\nTriplo homicídio na colônia (GDE)\nBarra do Ceará.\n\nhttps://t.me/Portalnoticiasceara/5719,Barra do Ceará,2024-12-14T07:50:04+00:00,None


In [27]:
df['bairro'].value_counts().head(20)

bairro
Barra do Ceará           21
Edson Queiroz            16
Vila Velha               14
Jangurussu               11
Barroso                  10
Conjunto Palmeiras        9
Quintino Cunha            9
Mondubim                  8
Planalto Ayrton Senna     8
Carlito Pamplona          8
Pici                      7
Pedras                    7
Messejana                 7
Presidente Vargas         7
Pirambu                   7
Santa Rosa                6
Jardim das Oliveiras      6
Serrinha                  5
Bom Jardim                5
Henrique Jorge            5
Name: count, dtype: int64

In [28]:
import zstandard
decompressor = zstandard.ZstdDecompressor()

In [29]:
def download_and_decompress(path, bucket_name):
    # Baixar o arquivo do MinIO
    result = minio_client.get_object(bucket_name, path)
    
    # Ler os dados comprimidos
    compressed_data = result.read()
    
    # Descomprimir os dados
    decompressor = zstandard.ZstdDecompressor()
    
    try:
        # Usando BytesIO para garantir que os dados sejam tratados como fluxo
        decompressed_data = decompressor.decompress(compressed_data)
        return decompressed_data
    except zstandard.ZstdError as e:
        print(f"Erro ao descomprimir o arquivo {path}: {e}")
        return None

In [30]:
paths = df[df['id'] == 5777]['media_path']  # Carregar os paths do dataframe

for path in paths:
    print(f"Processando {path}...")
    decompressed_data = download_and_decompress(path, bucket_name)
    
    if decompressed_data:
        print(f"Arquivo {path} descomprimido com sucesso.")
        # Aqui você pode salvar ou processar os dados descomprimidos
    else:
        print(f"Falha ao descomprimir o arquivo {path}.")

Processando None...


TypeError: 